In [ ]:
from pathlib import Path
import torch
from torch import nn
from mltools import ld, draw, models, MachineLearning

In [ ]:
train_iter, val_iter, test_iter, vocab = ld.chn_senti_corp('../data/ChnSentiCorp_htl_all.csv')  # 加载词表，数据集
rnn_layer = nn.LSTM(len(vocab), hidden_size=512, batch_first=True, bidirectional=True)  # 定义循环网络LSTM
model = models.RNNModel(rnn_layer, vocab_size=len(vocab), step_size=200, output_size=2)  # 定义训练模型
device = torch.device('cuda')


class LSTMCSCMachineLearning(MachineLearning):
    def calculate_model(self, x):
        '''计算神经网络'''
        y, _ = self.model(x)
        return y

    def grad_update(self, loss):
        '''梯度更新'''
        self.optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1, norm_type=2)
        self.optimizer.step()


ml = LSTMCSCMachineLearning(model, train_iter, val_iter, test_iter, device=device)
model_path = '../model/LSTM_CSC.pth'

In [ ]:
if True:
    ml.train(num_epochs=30, learning_rate=1e-1)  # 训练
    if True:
        torch.save(model.state_dict(), model_path)
elif Path(model_path).exists():
    model.load_state_dict(torch.load(model_path, map_location=device))

In [ ]:
ml.test()  # 测试

In [ ]:
# 预测模型
def predict(model, test_iter, device):
    model.to(device)  # 将网络复制到device上
    # 从测试中取一个批量
    x, y = next(iter(test_iter))
    x, y = x[:10].to(device), y[:10].to(device)
    values = [''.join(item) for item in vocab[x.tolist()]]
    # 预测
    y_pred, _ = model(x)
    y_pred = y_pred.argmax(dim=1)
    for pred, true, value in zip(y_pred, y, values):
        print(f'预测值 {pred}, 真实值 {true}, 数据 {value}')


predict(model, test_iter, device)